### 리뷰데이터 이미지 URL로 원본 사진 가져오기

In [1]:
import pandas as pd
import numpy as np

In [2]:
## CSV 파일 불러오기
df = pd.read_csv('C:\last_project\kakaomap_yeonnam_reviews_final(0530).csv') #, encoding='cp949')
df

,사업장명,분류,카카오맵ID,리뷰어이름,리뷰어레벨,후기개수,별점평균,팔로워수,작성일,별점,리뷰내용,사진URL
0,멜랑커피 연남,카페,180472632,entp,32,61,4.1,12,2025.05.28.,5.0,NaN,NaN
1,순대일번지,순대,821235252,댕,4,2,1.5,1,2025.05.07.,2.0,굳이... 평점이 왜 높은지 모르겠는... 밥도 설익고 식었고..국물이 그렇게 ...,NaN
2,순대일번지,순대,821235252,리브흐,21,47,5.0,0,2025.03.20.,5.0,가게이름이 납득이갑니다안짜고요 양많아요양념으로 범벅하는 순대국 아니라 좋아요,NaN
3,순대일번지,순대,821235252,마르스,32,80,3.9,25,2025.02.27.,4.0,순대국특 먹었어요.다대기 들어있으니까 일단 먹어보고 싱거우면 새우젖으로 간해서 드세...,//img1.kakaocdn.net/cthumb/local/C280x280.q50/...
4,순대일번지,순대,821235252,여행 지킴이,32,116,4.7,1,2025.02.10.,5.0,여행타임tv를 운영하는 크리에이터입니다서울시내 순대국집 100군데를 다녀본바로 여긴...,//img1.kakaocdn.net/cthumb/local/C280x280.q50/...
...,...,...,...,...,...,...,...,...,...,...,...,...
18997,코리아식당,한식,15742085,예민보,27,68,3.6,1,2019.05.05.,2.0,차돌박이 된장찌개 맹탕,NaN
18998,코리아식당,한식,15742085,Ml,25,146,3.9,2,2019.02.10.,5.0,가장 점심 먹고 싶은 집중 하나,NaN
18999,코리아식당,한식,15742085,ㅊㅇ,28,93,3.9,1,2018.08.07.,4.0,뭘 시켜도 맛있고 밥집으로도 술집으로도 훌륭함.곱창찌개랑 돈까스 두부김치 전도 맛있음,NaN
19000,코리아식당,한식,15742085,으뜸,22,70,4.0,1,2017.11.02.,5.0,점심 12시 넘으면 줄 서야해요. 이거저거 맛있는데 곱창찌개는 정말 맛있어요.,NaN


In [3]:
## URL 주소로 이미지 가져와 특정 폴더에 저장하기(수정본, 0529)

import os
import requests
import urllib.parse
import pandas as pd
import numpy as np
import re

## 이미지를 저장할 폴더 설정
# DOWNLOAD_DIR = "downloaded_images" # 이미지를 저장할 폴더 이름
DOWNLOAD_DIR = "yeonnam_review_images(0530)" # 이미지를 저장할 폴더 이름

## 폴더가 없으면 생성
if not os.path.exists(DOWNLOAD_DIR):
    os.makedirs(DOWNLOAD_DIR)
    print(f"'{DOWNLOAD_DIR}' 폴더를 생성했습니다.")

print(f"이미지를 '{DOWNLOAD_DIR}' 디렉토리에 다운로드합니다.")

## url 문자열 → 원본주소 디코딩 → 리스트 변환 함수
def urls_string_to_list(url_string):
    if pd.isna(url_string) or url_string == 'N/A':      # NaN 및 'N/A' 처리
        return []

    urls_list = []
    ## 쉼표로 분리 후 각 요소의 공백 제거, 그리고 빈 문자열 제거
    temp_url_list = [url.strip() for url in url_string.split(',') if url.strip()]
    
    for encoded_url in temp_url_list:
        # URL의 쿼리 파라미터 부분을 파싱
        # urlparse는 URL을 구성 요소로 분해, parse_qs는 쿼리 문자열을 딕셔너리로 변환
        parsed_url = urllib.parse.urlparse(encoded_url)
        query_params = urllib.parse.parse_qs(parsed_url.query)

        # query_params에서 'fname' 파라미터(리스트)의 첫 번째 요소를 가져옴
        if 'fname' in query_params and query_params['fname']:
            encoded_fname = query_params['fname'][0]

            # 가져온 'fname' 값을 URL 주소로 디코딩 및 리스트로 저장
            decoded_fname = urllib.parse.unquote(encoded_fname)
            urls_list.append(decoded_fname)
            # print(f"원본 URL: {url}")
            # print(f"디코딩된 이미지 경로: {decoded_fname}\n")
            
        else:
            print(f"URL에 'fname' 파라미터가 없거나 비어 있습니다: {url}\n")
            return []

    return urls_list

# MIME 타입에 따른 확장자 매핑 (더 많은 타입 추가 가능)
MIME_TYPE_TO_EXTENSION = {
    "image/jpeg": ".jpg",
    "image/png": ".png",
    "image/gif": ".gif",
    "image/bmp": ".bmp",
    "image/webp": ".webp",
}

## URL에서 이미지를 다운로드하고 Content-Type에 따라 자동으로 확장자를 붙여 저장하는 함수
def download_img_with_auto_extension(idx, image_url, download_folder, file_name):    
    try:
        # HTTP GET 요청으로 이미지 데이터 가져오기 (stream=True로 큰 파일 처리 효율화)
        response = requests.get(image_url, stream=True, timeout=5)
        response.raise_for_status() # HTTP 오류 (4xx, 5xx) 발생 시 예외 발생

        # Content-Type 헤더에서 MIME 타입 파악
        content_type = response.headers.get('Content-Type')
        
        # MIME 타입에 따라 확장자 결정
        extension = MIME_TYPE_TO_EXTENSION.get(content_type, ".bin") # 매핑되지 않으면 .bin으로 저장

        # 최종 저장 경로
        file_path = os.path.join(download_folder, f"{file_name}{extension}")

        # 이미지 데이터 저장
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        
        # print(f"✅ 이미지 저장 성공 '{image_url}' -> '{file_path}' (Content-Type: {content_type})")
        print(f"✅ {idx}번째 이미지 저장 성공 → '{file_path}'")
        return True

    except requests.exceptions.RequestException as e:
        print(f"❌ 이미지 저장 실패 ({image_url}): {e}")
        return False
    except Exception as e:
        print(f"❌ 예상치 못한 오류 발생 ({image_url}): {e}")
        return False

# 모든 이미지 다운로드 시도
for i in range(len(df)):
    image_urls_list = urls_string_to_list(df['사진URL'][i])
    if not image_urls_list:     # url 없음(빈 리스트)의 경우 다음으로 넘어감
        continue

    img_name_store = str(df['사업장명'][i].replace(' ', ''))
    img_name_kakaomapid = str(df['카카오맵ID'][i])
    # img_name_reviewer = 'na' if pd.isna(df['리뷰어이름'][i]) else str(df['리뷰어이름'][i]).replace(' ', '') # 빈칸이면 'na'
    img_name_reviewer = 'na' if pd.isna(df['리뷰어이름'][i]) else re.sub(r'[^가-힣a-zA-Z0-9]', '', str(df['리뷰어이름'][i])) # 빈칸이면 'na'

    print(f"\n상호명: '{img_name_store}', 리뷰어명: '{img_name_reviewer}'의 후기 이미지 {len(image_urls_list)}개")
    
    for idx, img_url in enumerate(image_urls_list, start=1):
        file_name = img_name_store + '_' + img_name_kakaomapid + '_' + img_name_reviewer + '_' + f'image{idx}'
        # print(f"[{idx+1}/{len(image_urls_list)}] 다운로드 시작: {img_url}")
        download_img_with_auto_extension(idx, img_url, DOWNLOAD_DIR, file_name)

print("\n✅✅ 모든 이미지 다운로드 완료 ✅✅")


'yeonnam_review_images(0530)' 폴더를 생성했습니다.
이미지를 'yeonnam_review_images(0530)' 디렉토리에 다운로드합니다.

상호명: '순대일번지', 리뷰어명: '마르스'의 후기 이미지 7개
✅ 1번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_마르스_image1.png'
✅ 2번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_마르스_image2.png'
✅ 3번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_마르스_image3.png'
✅ 4번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_마르스_image4.png'
✅ 5번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_마르스_image5.png'
✅ 6번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_마르스_image6.png'
✅ 7번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_마르스_image7.png'

상호명: '순대일번지', 리뷰어명: '여행지킴이'의 후기 이미지 9개
✅ 1번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_여행지킴이_image1.png'
✅ 2번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_여행지킴이_image2.png'
✅ 3번째 이미지 저장 성공 → 'yeonnam_review_images(0530)\순대일번지_821235252_여행지킴이_image3.png'
✅ 4번째 이미지 저장 성공 → 'yeonnam_review_